In [15]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [16]:
from scipy.interpolate import interp1d
# resample target
def resample_cycle(cycle, num_points):
    """Resamples a gait cycle to have a fixed number of points using interpolation."""
    original_indices = np.linspace(0, 1, len(cycle))  # original indices based on the length of the cycle
    new_indices = np.linspace(0, 1, num_points)  # new indices to match the desired number of points
    interpolation_function = interp1d(original_indices, cycle, kind='linear')  # interpolate the data
    return interpolation_function(new_indices)  # return the resampled cycle

def rotate_cyclic(df, column_name, range):
    # Move the last value to the first position using pd.concat()
    return pd.concat([df[column_name].iloc[-range:], df[column_name].iloc[:-range]]).reset_index(drop=True)


In [17]:
leg_0_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_withz_me_0.csv')
leg_1_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_withz_me_1.csv')
leg_2_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_withz_me_2.csv')

# leg_0_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_withz_si_0.csv')
# leg_1_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_withz_si_1.csv')
# leg_2_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_withz_si_2.csv')

# leg_0_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_withz_ma21_0.csv')
# leg_1_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_withz_ma21_1.csv')
# leg_2_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_withz_ma21_2.csv')
leg_0_df.rename(columns={'tr': 'TR0', 'cr': 'CR0', 'fr': 'FR0', 'z': 'Z0'}, inplace=True)
leg_1_df.rename(columns={'tr': 'TR1', 'cr': 'CR1', 'fr': 'FR1', 'z': 'Z1'}, inplace=True)
leg_2_df.rename(columns={'tr': 'TR2', 'cr': 'CR2', 'fr': 'FR2', 'z': 'Z2'}, inplace=True)

leg_0_df['TR0'] = rotate_cyclic(leg_0_df, 'TR0', 50)
leg_0_df['CR0'] = rotate_cyclic(leg_0_df, 'CR0', 50)
leg_0_df['FR0'] = rotate_cyclic(leg_0_df, 'FR0', 50)
leg_0_df['Z0'] = rotate_cyclic(leg_0_df, 'Z0',   50)
leg_1_df['TR1'] = rotate_cyclic(leg_1_df, 'TR1', 50)
leg_1_df['CR1'] = rotate_cyclic(leg_1_df, 'CR1', 50)
leg_1_df['FR1'] = rotate_cyclic(leg_1_df, 'FR1', 50)
leg_1_df['Z1'] = rotate_cyclic(leg_1_df, 'Z1',   50)
leg_2_df['TR2'] = rotate_cyclic(leg_2_df, 'TR2', 50)
leg_2_df['CR2'] = rotate_cyclic(leg_2_df, 'CR2', 50)
leg_2_df['FR2'] = rotate_cyclic(leg_2_df, 'FR2', 50)
leg_2_df['Z2'] = rotate_cyclic(leg_2_df, 'Z2',   50)


# leg_0_df['F0'] = np.where(leg_0_df['Z0'] > 0.0, 0.0, 1.0)
# leg_1_df['F1'] = np.where(leg_1_df['Z1'] > 0.0, 0.0, 1.0)
# leg_2_df['F2'] = np.where(leg_2_df['Z2'] > 0.0, 0.0, 1.0)


# Apply the condition for leg_0_df
leg_0_df['F0'] = leg_0_df['Z0'].apply(lambda x: 0.0 if x > 0.0 else 1.0)
leg_1_df['F1'] = leg_1_df['Z1'].apply(lambda x: 0.0 if x > 0.0 else 1.0)
leg_2_df['F2'] = leg_2_df['Z2'].apply(lambda x: 0.0 if x > 0.0 else 1.0)

window_size = 50

# Apply the moving average for each dataframe
leg_0_df['F0'] = leg_0_df['F0'].rolling(window=window_size, min_periods=1).mean()
leg_1_df['F1'] = leg_1_df['F1'].rolling(window=window_size, min_periods=1).mean()
leg_2_df['F2'] = leg_2_df['F2'].rolling(window=window_size, min_periods=1).mean()


fig = go.Figure()
fig = make_subplots(rows=4, cols=3, shared_xaxes=False)
fig.update_layout(height=1000, width=1100, title="Path plot")
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))  # Reduce margins

fig.add_trace(go.Scatter( y=leg_0_df['TR0'], mode='lines', name=f'TR0'), row=1, col=1)
fig.add_trace(go.Scatter( y=leg_0_df['CR0'], mode='lines', name=f'CR0'), row=2, col=1)
fig.add_trace(go.Scatter( y=leg_0_df['FR0'], mode='lines', name=f'FR0'), row=3, col=1)
fig.add_trace(go.Scatter( y=leg_0_df['Z0'], mode='lines', name=f'Z0'), row=4, col=1)
fig.add_trace(go.Scatter( y=leg_0_df['F0'], mode='lines', name=f'F0'), row=4, col=1)

fig.add_trace(go.Scatter( y=leg_1_df['TR1'], mode='lines', name=f'TR1'), row=1, col=2)
fig.add_trace(go.Scatter( y=leg_1_df['CR1'], mode='lines', name=f'CR1'), row=2, col=2)
fig.add_trace(go.Scatter( y=leg_1_df['FR1'], mode='lines', name=f'FR1'), row=3, col=2)
fig.add_trace(go.Scatter( y=leg_1_df['Z1'], mode='lines', name=f'Z1'), row=4, col=2)
fig.add_trace(go.Scatter( y=leg_1_df['F1'], mode='lines', name=f'F1'), row=4, col=2)

fig.add_trace(go.Scatter( y=leg_2_df['TR2'], mode='lines', name=f'TR2'), row=1, col=3)
fig.add_trace(go.Scatter( y=leg_2_df['CR2'], mode='lines', name=f'CR2'), row=2, col=3)
fig.add_trace(go.Scatter( y=leg_2_df['FR2'], mode='lines', name=f'FR2'), row=3, col=3)
fig.add_trace(go.Scatter( y=leg_2_df['Z2'], mode='lines', name=f'Z2'), row=4, col=3)
fig.add_trace(go.Scatter( y=leg_2_df['F2'], mode='lines', name=f'F2'), row=4, col=3)

In [18]:
print(leg_0_df.keys())

Index(['TR0', 'CR0', 'FR0', 'Z0', 'F0'], dtype='object')


In [19]:
from include.cpg_rbf.cpg_so2 import CPG_SO2
# from include.cpg_rbf.cpg_afdc import CPG_AFDC
from include.cpg_rbf.rbf import RBF

rbf = RBF()
cpg = CPG_SO2()
cpg_one_cycle = cpg.generate_cpg_one_cycle(0.05)    # lowest phi value
out0_cpg_one_cycle = cpg_one_cycle['out0_cpg_one_cycle'][:] # Use full cycle of CPG
out1_cpg_one_cycle = cpg_one_cycle['out1_cpg_one_cycle'][:] # Use full cycle of CPG
cpg_cycle_length = len(out0_cpg_one_cycle)
cpg_kernel = rbf.construct_kernels_with_cpg_one_cycle(out0_cpg_one_cycle, out1_cpg_one_cycle, cpg_cycle_length) # construct kernels with cpg one cycle

In [20]:
# Define dataframes and corresponding column groups
dataframes = [leg_0_df, leg_1_df, leg_2_df]
column_groups = [
    ['TR0', 'CR0', 'FR0', 'Z0'],
    ['TR1', 'CR1', 'FR1', 'Z1'],
    ['TR2', 'CR2', 'FR2', 'Z2']
]

imitated_paths = {}
imitated_weights = {}

# Loop through each dataframe and corresponding column group
for df, columns in zip(dataframes, column_groups):
    for col in columns:
        # Resample cycle for the current column
        target = resample_cycle(np.array(df[col]), cpg_cycle_length)
        
        # Imitate path and get weights using RBF
        imitated_paths[col] = rbf.imitate_path_by_learning(target)
        imitated_weights[col] = rbf.get_rbf_weight()
        



last iteration: 11
last iteration: 9
last iteration: 15
last iteration: 13
last iteration: 9
last iteration: 13
last iteration: 42
last iteration: 15
last iteration: 11
last iteration: 12
last iteration: 18
last iteration: 13


In [21]:
rbf_f = RBF()
out0_cpg_one_cycle_f = cpg_one_cycle['out0_cpg_one_cycle'][:] # Use full cycle of CPG
out1_cpg_one_cycle_f = cpg_one_cycle['out1_cpg_one_cycle'][:] # Use full cycle of CPG
cpg_cycle_length_f = len(out0_cpg_one_cycle_f)
cpg_kernel_f = rbf_f.construct_kernels_with_cpg_one_cycle(out0_cpg_one_cycle_f, out1_cpg_one_cycle_f, cpg_cycle_length_f) # construct kernels with cpg one cycle

In [22]:
imitated_force = {}
imitated_force_weights = {}
column_groups = [
    ['F0'],
    ['F1'],
    ['F2']
]
for df, columns in zip(dataframes, column_groups):
    for col in columns:
        # Resample cycle for the current column
        target = resample_cycle(np.array(df[col][:]), cpg_cycle_length_f)
        
        # Imitate path and get weights using RBF
        imitated_force[col] = rbf_f.imitate_path_by_learning(target)
        imitated_force_weights[col] = rbf_f.get_rbf_weight()
print(imitated_force_weights)

last iteration: 341
last iteration: 344
last iteration: 77
{'F0': array([ 1.50509691e-03,  2.95803788e-01,  4.06366584e-01, -3.64634607e-01,
       -7.09860536e-01, -1.06199443e-01,  4.89554385e-01,  1.15002195e+00,
       -5.06729279e-02, -3.48717240e-01, -9.13953731e-02,  2.27968950e-01,
        2.52735572e-01,  1.85062865e-01,  2.41138105e-01,  2.78053519e-01,
        3.71795361e-02, -2.74745885e-01, -2.07430564e-01,  8.00027118e-01,
        9.32873916e-01,  4.44320763e-02, -5.59504150e-01, -9.00257205e-01,
        2.81632240e-01,  5.00235137e-01,  1.63541014e-01, -2.07494090e-01,
       -1.44269589e-01, -1.14823014e-02,  8.26277964e-02,  5.99298309e-02,
       -7.31301704e-03, -2.98071203e-02, -2.16741225e-02,  5.87425195e-04,
        7.35890865e-03,  5.66976627e-03,  4.04376192e-03,  5.66003776e-03,
        3.52087520e-03, -1.30058824e-02, -2.60112321e-02, -1.28978643e-02,
        3.95920194e-02,  6.56724083e-02,  1.95510815e-02, -7.45849391e-02,
       -1.86017246e-01, -8.5053185

In [23]:

fig = go.Figure()
fig = make_subplots(rows=6, cols=1, shared_xaxes=False)
fig.update_layout(height=800, width=1100, title="CPG plot")
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))  # Reduce margins

# for i in range(len(cpg_kernel)):
#     fig.add_trace(go.Scatter( y=cpg_kernel[i], mode='lines', name=f'kernels{i}'), row=2, col=1)

fig.add_trace(go.Scatter( y=out0_cpg_one_cycle, mode='lines', name=f'out0_cpg_one_cycle'), row=1, col=1)
fig.add_trace(go.Scatter( y=out1_cpg_one_cycle, mode='lines', name=f'out1_cpg_one_cycle'), row=1, col=1)

fig.add_trace(go.Scatter( y=imitated_paths['TR0'], mode='lines', name=f'TR0 (CPG-RBF)'), row=3, col=1)
fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_0_df['TR0']), cpg_cycle_length), mode='lines', name=f'TR0'), row=3, col=1)

fig.add_trace(go.Scatter( y=imitated_paths['CR0'], mode='lines', name=f'CR0 (CPG-RBF)'), row=4, col=1)
fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_0_df['CR0']), cpg_cycle_length), mode='lines', name=f'CR0'), row=4, col=1)

fig.add_trace(go.Scatter( y=imitated_paths['FR0'], mode='lines', name=f'FR0 (CPG-RBF)'), row=5, col=1)
fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_0_df['FR0']), cpg_cycle_length), mode='lines', name=f'FR0'), row=5, col=1)

fig.add_trace(go.Scatter( y=imitated_force['F0'], mode='lines', name=f'F0 (CPG-RBF)'), row=6, col=1)
fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_0_df['F0']), cpg_cycle_length), mode='lines', name=f'F0'), row=6, col=1)

# fig.add_trace(go.Scatter( y=imitated_paths['TR1'], mode='lines', name=f'TR1 (CPG-RBF)'), row=3, col=1)
# fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_1_df['TR1']), cpg_cycle_length), mode='lines', name=f'TR1'), row=3, col=1)

# fig.add_trace(go.Scatter( y=imitated_paths['CR1'], mode='lines', name=f'CR1 (CPG-RBF)'), row=4, col=1)
# fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_1_df['CR1']), cpg_cycle_length), mode='lines', name=f'CR1'), row=4, col=1)

# fig.add_trace(go.Scatter( y=imitated_paths['FR1'], mode='lines', name=f'FR1 (CPG-RBF)'), row=5, col=1)
# fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_1_df['FR1']), cpg_cycle_length), mode='lines', name=f'FR1'), row=5, col=1)

In [24]:
np.savez('imitated_stick_insect_right_leg_weights_me.npz', **imitated_weights)
np.savez('imitated_stick_insect_right_leg_weights_me_force.npz', **imitated_force_weights)

In [25]:
# imitated_weights = np.load('imitated_stick_insect_right_leg_weights_me.npz')
# print(imitated_weights.files)

# # Access each array by its key
# for key in imitated_weights.files:
#     print(f"{key}: {imitated_weights[key]}")

In [26]:
# cpg = CPG_SO2()
# cpg_output_0 = []
# cpg_output_1 = []
# online_imitated_paths = {'TR0':[], 'CR0':[], 'FR0':[], 'Z0':[], 'F0':[]}
# for i in range(1000):
#     cpg.s = 1.0
#     cpg.update_cpg(0.05)      # phi would be equal when learning


#     cpg_output_0.append(cpg.out0_t)
#     cpg_output_1.append(cpg.out1_t)
    
#     online_imitated_paths['TR0'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['TR0']))
#     online_imitated_paths['CR0'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['CR0']))
#     online_imitated_paths['FR0'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['FR0']))
#     online_imitated_paths['F0'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['F0']))

# fig = go.Figure()
# fig = make_subplots(rows=5, cols=1, shared_xaxes=True)
# fig.update_layout(height=600, width=1100, title="CPG plot")
# fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))  # Reduce margins

# fig.add_trace(go.Scatter( y=cpg_output_0, mode='lines', name=f'cpg_output_0'), row=1, col=1)
# fig.add_trace(go.Scatter( y=cpg_output_1, mode='lines', name=f'cpg_output_1'), row=1, col=1)
# fig.add_trace(go.Scatter( y=online_imitated_paths['TR0'], mode='lines', name=f'TR0 (CPG-RBF)'), row=2, col=1)
# fig.add_trace(go.Scatter( y=online_imitated_paths['CR0'], mode='lines', name=f'CR0 (CPG-RBF)'), row=3, col=1)
# fig.add_trace(go.Scatter( y=online_imitated_paths['FR0'], mode='lines', name=f'FR0 (CPG-RBF)'), row=4, col=1)
# fig.add_trace(go.Scatter( y=online_imitated_paths['F0'], mode='lines', name=f'F0 (CPG-RBF)'), row=5, col=1)




In [27]:
# from include.cpg_rbf.cpg_so2 import CPG_SO2
# # from include.cpg_rbf.cpg_afdc import CPG_AFDC
# from include.cpg_rbf.rbf import RBF

# rbf = RBF()
# cpg = CPG_SO2()
# cpg_one_cycle = cpg.generate_cpg_one_cycle(0.05)    # lowest phi value
# out0_cpg_one_cycle = cpg_one_cycle['out0_cpg_one_cycle'][:] # Use full cycle of CPG
# out1_cpg_one_cycle = cpg_one_cycle['out1_cpg_one_cycle'][:] # Use full cycle of CPG
# cpg_cycle_length = len(out0_cpg_one_cycle)
# cpg_kernel = rbf.construct_kernels_with_cpg_one_cycle(out0_cpg_one_cycle, out1_cpg_one_cycle, cpg_cycle_length) # construct kernels with cpg one cycle

In [28]:
# cpg = CPG_SO2()
# cpg_output_0 = []
# cpg_output_1 = []
# online_imitated_paths = {'TR1':[], 'CR1':[], 'FR1':[], 'Z1':[], 'F1':[]}
# for i in range(1000):
#     cpg.s = 1.0
#     cpg.update_cpg(0.05)      # phi would be equal when learning


#     cpg_output_0.append(cpg.out0_t)
#     cpg_output_1.append(cpg.out1_t)
    
#     online_imitated_paths['TR1'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['TR1']))
#     online_imitated_paths['CR1'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['CR1']))
#     online_imitated_paths['FR1'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['FR1']))
#     online_imitated_paths['F1'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_force_weights['F1']))

# fig = go.Figure()
# fig = make_subplots(rows=5, cols=1, shared_xaxes=True)
# fig.update_layout(height=600, width=1100, title="CPG plot")
# fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))  # Reduce margins

# fig.add_trace(go.Scatter( y=cpg_output_0, mode='lines', name=f'cpg_output_0'), row=1, col=1)
# fig.add_trace(go.Scatter( y=cpg_output_1, mode='lines', name=f'cpg_output_1'), row=1, col=1)
# fig.add_trace(go.Scatter( y=online_imitated_paths['TR1'], mode='lines', name=f'TR0 (CPG-RBF)'), row=2, col=1)
# fig.add_trace(go.Scatter( y=online_imitated_paths['CR1'], mode='lines', name=f'CR0 (CPG-RBF)'), row=3, col=1)
# fig.add_trace(go.Scatter( y=online_imitated_paths['FR1'], mode='lines', name=f'FR0 (CPG-RBF)'), row=4, col=1)
# fig.add_trace(go.Scatter( y=online_imitated_paths['F1'], mode='lines', name=f'F0 (CPG-RBF)'), row=5, col=1)
